# Assignment 2 Addendum

In [2]:
import numpy as np

class Bond:
    
    def __init__(self, maturity: int, rate: float, coupon: float, face: float = 1000., frequency: int = 2):
        self.maturity = maturity
        self.coupon = coupon
        self.rate = rate
        self.face = face
        self.frequency = frequency
        
        self.cash_flows = self.create_flows(maturity, coupon, face, frequency)
        self.pv = self.present_value(rate,self.cash_flows)
        
    @classmethod
    def no_pv(cls, maturity: int, rate: float, coupon: float, face: float = 1000., frequency: int = 2):
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_ytm(cls, maturity: int, pv: float, coupon: float, face: float = 1000., frequency: int = 2):
        
        cls.frequency = frequency
        rate = cls.ytm(cls,pv,cls.create_flows(cls, maturity, coupon, face, frequency))
        
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_coupon(cls, maturity: int, pv: float, rate: float, face: float = 1000., frequency: int = 2):
        
        cls.frequency = frequency
        coupon = cls.coupon_rate(cls,maturity,pv,rate,face,frequency)
        
        return cls(maturity, rate, coupon, face, frequency)
    
    @classmethod
    def no_face(cls, maturity: int, pv: float, rate: float, coupon: float, frequency: int = 2):
    
        cls.frequency = frequency
        face = cls.face_value(cls,maturity,pv,rate,coupon,frequency)
        
        return cls(maturity, rate, coupon, face, frequency)
    
    def __str__(self):
        return f'a $%.02f, %.02f percent coupon bond with %d years to maturity' % (self.pv,self.coupon*100,self.maturity)

    def create_flows(self, maturity: int, coupon: float, face: float, frequency: int) -> np.ndarray:
        pmt = coupon * face / frequency
        bond = np.full(maturity*frequency,pmt)
        bond[-1] += face
        return bond
            
    def present_value(self, rate: float, cash_flows: np.ndarray) -> float:
        pv = 0
        for count, flow in enumerate(cash_flows):
            pv += flow/((1+rate/self.frequency)**(count+1))
        return pv
    
    def ytm(self, pv: float, cash_flows: np.ndarray, lower: float = 0., upper: float = 1.) -> float:

        while(True): 
            guess = (lower + upper)/2
            #      Guessed price      -       Real price
            diff = self.present_value(self, guess, cash_flows) - pv
            if (abs(diff) < 0.00001):
                return guess
            else:
                if (diff > 0): # The guessed price is too high, so our guess is too low
                    lower = guess # Since our guess is too low, we raise the lower limit of our search
                else:
                    upper = guess
        
    def coupon_rate(self, maturity: int, pv: float, rate: float, face: float, frequency: int):
        
        factors = np.ones(maturity*frequency)
        coupon = (pv - face/(1+rate/frequency)**(maturity*frequency))
        coupon /= self.present_value(self,rate,factors)
        coupon /= face
        coupon *= frequency
        
        return coupon
    
    def face_value(self, maturity: int, pv: float, rate: float, coupon: float, frequency: int):
        
        coupons = self.create_flows(self,maturity,coupon,1,frequency)
        face = pv / self.present_value(self,rate,coupons)
        
        return face

## Problem 1

What is the price of a 10-year, zero-coupon bond paying \$1, 000 at maturity
if the YTM is:

a. 5 percent ?
b. 10 percent ?
c. 15 percent ?

In [7]:
li = ['a','b','c']
rates = [.05,.1,.15]
lists = [li,rates]; tuples = zip(*lists)
for letter, rate in tuples:
    print(f'{letter}. The bond price is ${Bond(10,rate,0.,frequency=1).pv:.02f}')

a. The bond price is $613.91
b. The bond price is $385.54
c. The bond price is $247.18


## Problem 2

Calculate the price of 25-year, 7 percent coupon bond with a \$1,000 par value that makes semiannual payments with a YTM of:

a. 7 percent
b. 9 percent
c. 5 percent

In [9]:
li = ['a','b','c']; rates = [.07,.09,.05]
lists = [li,rates]; tuples = zip(*lists)
for letter, rate in tuples:
    print(f'{letter}. The bond price is ${Bond(25,rate,.07).pv:.02f}')

a. The bond price is $1000.00
b. The bond price is $802.38
c. The bond price is $1283.62


## Problem 3
Watters Umbrella Corp. issued 12-year bonds 2 years ago at a coupon rate
of 7.8 percent. The bonds make semiannual payments. If these bonds
currently sell for 105 percent of par value, what is the YTM?

In [10]:
print(f'The YTM of the bond is {Bond.no_ytm(10,1.05*1000,.078).rate*100:.02f}%.')

The YTM of the bond is 7.09%.


## Problem 4
Hacker Software has 7.4 percent coupon bonds on the market with 9 years
to maturity. The bonds make semiannual payments and currently sell for
96 percent of par. What is the YTM?

In [11]:
print(f'The YTM of the bond is {Bond.no_ytm(9,.96*1000,.074).rate*100:.02f}%.')

The YTM of the bond is 8.03%.


## Problem 5
Pembroke Co. wants to issue new 20-year bonds for some much needed
expansion projects. The company currently has 10 percent coupon bonds
on the market that sell for $1, 063, make semiannual payments, and mature
in 20 year. What coupon rate should the company set on its new bonds if
it wants them to sell at par?

Answer: To sell at par, the coupon rate needs to be equal to the YTM and we can assume that Pembroke Co. wants the cost of debt of the new bonds to match the cost of the old bonds.

In [16]:
old_bond = Bond.no_ytm(20,1063,.1)
new_bond = Bond.no_coupon(20,1000,old_bond.rate)
print(old_bond.rate)
print(new_bond.coupon)

0.0930052399635315
0.09300523996353148


## Additional Present Value Problems
Jimbo has \\$2,780 to invest. He wants to invest the most he can into a combination of two bonds. The two bonds have an identical YTM of 4.3% and a face value of \\$1,000 and both pay semiannual coupons. The first bond has a coupon rate of 4.7 percent and 10 years to maturity and the second has a coupon rate of 3.5 percent and 12 years to maturity. What is the best combination of the two bonds and what is the maximum amount Jimbo can invest into them?

In [17]:
import math
bond1 = Bond(10,.043,.047)
bond2 = Bond(12,.043,.035)
best = [0,0]
big = 0
for i in range(math.ceil(2780 / bond1.pv)+1):
    for j in range(math.ceil(2780 / bond2.pv)+1):
        value = bond1.pv * i + bond2.pv * j
        if ((value > big) & (value <= 2780)):
            big = value
            best[0] = i
            best[1] = j
print(best,big)

[0, 3] 2776.8427826744596


## Additional YTM Problems
Geoffrey wants to invest in bonds with a YTM of around 6%. Should he choose AlphaBravo, Inc.'s (\\$1,000 par, 10-year, 5 percent coupon rate with annual payments, and current price of \\$980.80) or CharlieDelta, LLC's (\\$1,000 par, 8-year, 5 percent coupon rate with semiannual payments, and current price of \\$917.92)?

In [21]:
bond_ab = Bond.no_ytm(10,980.80,.05,frequency=1)
bond_ab.name = 'AlphaBeta'

bond_cd = Bond.no_ytm(8,917.92,.05)
bond_cd.name = 'CharlieDelta'

if (abs(bond_ab.rate - .06) < abs(bond_cd.rate - .06)):
    closer = bond_ab
    farther = bond_cd
else:
    closer = bond_cd
    farther = bond_ab
    
print(f'Geoffrey should choose {closer.name} because its YTM of {closer.rate*100:.02f}% is closer to 6% than {farther.name}\'s YTM of {farther.rate*100:.02f}%.')

Geoffrey should choose CharlieDelta because its YTM of 6.32% is closer to 6% than AlphaBeta's YTM of 5.25%.
